In [269]:
import numpy as np
import sympy as symp
import scipy as scip
import matplotlib.pyplot as plt
import math as math

In [270]:
# Given constants
Pc = 30e5  # Chamber total pressure in Pa (30 bar)
dt = 0.0254  # Throat diameter in meters (1 inch) -- Assumed
Tc = 2500  # Chamber total temperature in Kelvin -- Assumed
gamma = 1.2  # Specific heat ratio -- Assumed
R = 300  # Specific gas constant in J/(kg·K) -- Assumed (approximated from 287 to 350)
OF = 6 # O/F Ratio
Pe = 0 # Exhaust exit pressure in Pa (Ideal engine in vacuum = 0 Pa)
Pa = 0 # Atmospheric pressure in Pa (Vacuum = 0)
g0 = 9.807 # Earth's gravitational acceleration at surface
mass_dry = 100  # in kg
nozzle_expansion_ratio = 125
density = 1600 # Propellant density in kg/m^3
h1 = 200e3  # Initial orbit altitude (200 km above Earth's surface)
h2 = 35786e3  # Geostationary orbit altitude (35,786 km above Earth's surface)

dt2 = math.sqrt(nozzle_expansion_ratio*dt) # Nozzle exit diameter

print(f"Chamber pressure = {Pc*1e-6:.2f} MPa")
print(f"Chamber temperature = {Tc} K")
print(f"Throat diameter = {dt*1000} mm")
print(f"Nozzle expansion ratio = {nozzle_expansion_ratio}")

# Step 1: Calculate Throat Area (At)
At = math.pi * (dt / 2) ** 2  # Throat area in square meters
Ae = nozzle_expansion_ratio*At # Nozzle exit area in square meters

print(f"Nozzle exit diameter = {dt2:.4f} m")
print(f"Nozzle exit area = {Ae:.4f} m^2")

# Step 2: Calculate Effective Exhaust Velocity (ve) and Specific Impulse (ISP)
def exit_mach_number_func(M, Ae, At, gamma):
    return (Ae/At - (1 / M) * ((2 / (gamma + 1)) * (1 + (gamma - 1) / 2 * M**2))**((gamma + 1) / (2 * (gamma - 1))))
Me = float(scip.optimize.fsolve(exit_mach_number_func, 5, args=(Ae, At, gamma))[0])
#ve = math.sqrt(2*gamma*R*Tc/(gamma-1)*(1-(Pe/Pc)**((gamma-1)/gamma)))
Te = Tc*(1+(gamma-1)/2*Me**2)**-1
Pe = Pc*(1+(gamma-1)/2*Me**2)**(-gamma/(gamma-1))
ve = Me*math.sqrt(gamma*R*Te)
ISP = ve/g0

print(f"\nExhaust mach number = {Me:.4f}")
print(f"Exhaust velocity = {ve:.4f} m/s")
print(f"Exhaust temperature = {Te:.4f} K")
print(f"Exhaust pressure = {Pe:.4f} Pa")
print(f"Specific Impulse = {ISP:.4f} s")
print(f"Density Impulse = {ISP*density:.4f} kg-s/m^3")

# Step 3: Calculate the mass flow rate (dot_m)
mass_flow_rate = (Pc * At / math.sqrt(Tc)) * math.sqrt(gamma / R) * ((2 / (gamma + 1)) ** ((gamma + 1) / (2 * (gamma - 1))))
oxidizer_mass_flow_rate = mass_flow_rate*OF/(1+OF)
fuel_mass_flow_rate = mass_flow_rate-oxidizer_mass_flow_rate

print(f"\nPropellant Mass Flow Rate = {mass_flow_rate:.4f} Kg/s")  # in kg/s
print(f"Fuel Mass Flow Rate = {fuel_mass_flow_rate:.4f} Kg/s")  # in kg/s
print(f"Oxidizer Mass Flow Rate = {oxidizer_mass_flow_rate:.4f} Kg/s")   # in kg/s

# Step 4: Calculate Thrust (Ft)
thrust = mass_flow_rate*ve+(Pe-Pa)*Ae

print(f"\nThrust = {thrust:.4f} N")

# Step 5: Delta V Calculation
R_E = 6378e3  # Earth's radius in meters
mu = 3.985894e14  # Standard gravitational parameter for Earth in m^3/s^2
g0 = 9.81  # Gravitational acceleration in m/s^2

# Initial and final orbital radii
r1 = R_E + h1
r2 = R_E + h2

# Semi-major axis of the transfer orbit
a_t = (r1 + r2) / 2

# Orbital velocities
v1 = math.sqrt(mu / r1)  # Initial orbit velocity
vp = math.sqrt((2 * mu / r1) - (mu / a_t))  # Velocity at perigee of transfer orbit
va = math.sqrt((2 * mu / r2) - (mu / a_t))  # Velocity at apogee of transfer orbit
v2 = math.sqrt(mu / r2)  # Final orbit velocity

# Delta V for each burn
delta_v1 = vp - v1
delta_v2 = v2 - va

# Total Delta V
delta_v_total = delta_v1 + delta_v2

# Using Tsiolkovsky's rocket equation to find propellant mass
mass_propellant = mass_dry * (math.exp(delta_v_total / ve) - 1)
burn_time = mass_propellant/mass_flow_rate

print(f"\nDelta V total = {delta_v_total:.4f} m/s")
print(f"Propellant mass required = {mass_propellant:.4f} kg")
print(f"Total LEO wet mass = {mass_propellant+mass_dry:.4f} kg")
print(f"Dry mass = {mass_propellant+mass_dry:.4f} kg")
print(f"Burn time = {burn_time:.2f} s")


Chamber pressure = 3.00 MPa
Chamber temperature = 2500 K
Throat diameter = 25.4 mm
Nozzle expansion ratio = 125
Nozzle exit diameter = 1.7819 m
Nozzle exit area = 0.0633 m^2

Exhaust mach number = 5.0524
Exhaust velocity = 2542.9732 m/s
Exhaust temperature = 703.6909 K
Exhaust pressure = 1492.0139 Pa
Specific Impulse = 259.3018 s
Density Impulse = 414882.9528 kg-s/m^3

Propellant Mass Flow Rate = 1.1384 Kg/s
Fuel Mass Flow Rate = 0.1626 Kg/s
Oxidizer Mass Flow Rate = 0.9757 Kg/s

Thrust = 2989.3152 N

Delta V total = 3931.8569 m/s
Propellant mass required = 369.3438 kg
Total LEO wet mass = 469.3438 kg
Dry mass = 469.3438 kg
Burn time = 324.45 s


In [271]:
#Delta V Calculation
